In [1]:
from util import *
import gc

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("new_test.csv")
all_data = pd.concat([train_data, test_data], ignore_index = True)

In [3]:
date_info = pd.read_csv("date_info.csv")
days = pd.Series([31,31,28,31,30,31,30,31,31,30,31,30,31])

In [4]:
date_info['visit_date2'] = pd.to_datetime(date_info['calendar_date'])
date_info['quarter'] = date_info['visit_date2'].dt.quarter
date_info.drop("visit_date2", axis = "columns", inplace = True)

In [5]:
date_info["week_end"] = 0
date_info.loc[date_info["day_of_week"]  == "Sunday", "holiday_flg"] = 1
date_info.loc[date_info["day_of_week"]  == "Friday", "week_end"] = 1
date_info.loc[date_info["day_of_week"]  == "Saturday","week_end"] = 1
date_info.columns  = ["date","week_day","holiday","quarter","week_end"]

In [6]:
#load and combine date features
date_info["date"]  =  date_info["date"].apply(get_gen_date)
date_info["week"]  =  date_info["date"].apply(get_week)

date_info["month"] =  date_info["date"].apply(lambda x : x.split("/")[0]).astype(np.int8)
date_info["year"]  =  date_info["date"].apply(lambda x : x.split("/")[2]).astype(np.int16)
date_info["day"]   =  date_info["date"].apply(lambda x : x.split("/")[1]).astype(np.int8)

date_info["days"]  =  date_info["month"].map(days).astype(np.int8)

agg = gb_util(date_info,["year","month"],"holiday","month_holiday_agg")
date_info = date_info.merge(agg,on=["year","month"],how="left")

agg = gb_util(date_info,["year","month","week"],"holiday","month_week_holiday_agg")
date_info = date_info.merge(agg,on=["year","month","week"],how="left")

In [7]:
date_info["month_num"] = 0
num = 1
for i in range(1, 13):
    date_info.loc[(date_info.month == i) & (date_info.year == 2016), "month_num"] = num
    num += 1
    
for i in range(1, 6):
    date_info.loc[(date_info.month == i) & (date_info.year == 2017), "month_num"] = num
    num += 1

In [8]:
num = 1
for i in range(1, date_info.month_num.max()+1):
    for j in range(1, date_info[date_info.month_num == i].week.max() + 1):
        date_info.loc[(date_info.month_num == i) & (date_info.week == j), "week_num"] = num
        num += 1 
    

In [9]:
week_map = {"Sunday":0,"Monday":1,"Tuesday":2,"Wednesday":3,"Thursday":4,"Friday":5,"Saturday":6}
date_info["week_day"] = date_info["week_day"].map(week_map)
date_info["week_day"] = date_info["week_day"].apply(int)

In [10]:
all_data["date"] = all_data["visit_date"]
all_data.drop("visit_date",axis = "columns",inplace = True)
all_data = all_data.merge(date_info,how="left",on=["date"])

In [11]:
Q1 = all_data["visitors"].quantile(0.01)
Q3 = all_data["visitors"].quantile(0.99)
IQR = Q3 - Q1
all_data.loc[all_data["visitors"] > (Q3 + 3 * IQR), "visitors"] = Q3 + 3 * IQR

df_temp = all_data.groupby(["chw_store_id"], as_index = False).agg({"visitors":"mean"})
df_temp.columns = ["chw_store_id", "vis"]
all_data = all_data.merge(df_temp, on = "chw_store_id", how = "left")
all_data.loc[all_data["visitors"] == 0, "visitors"] = all_data.loc[all_data["visitors"] == 0, "vis"]
all_data.drop("vis", inplace = True, axis = "columns")

In [12]:
all_data["min_date"] = pd.to_datetime(all_data["date"])
df_temp = all_data[["chw_store_id", "min_date", "visitors"]].copy()
df_temp.sort_values(by = "min_date", inplace = True)

In [13]:
df_temp.drop("min_date", inplace = True, axis = "columns")
all_data.drop("min_date", inplace = True, axis = "columns")

print("calculating median")
median_df = df_temp.groupby(["chw_store_id"],sort = False).expanding().median().reset_index()
print("calculating mean")
mean_df = df_temp.groupby(["chw_store_id"], sort = False).expanding().mean().reset_index()

medians = np.zeros((all_data.shape[0]))
means = np.zeros((all_data.shape[0]))
medians[median_df["level_1"].values] = median_df["visitors"].values
means[mean_df["level_1"].values] = mean_df["visitors"].values
all_data["median"] = medians
all_data["mean"] = means

calculating median
calculating mean


In [14]:
#########chwiggy store info#######
ch_store_info = pd.read_csv("chw_store_info.csv")
all_data = all_data.merge(ch_store_info,on=["chw_store_id"],how = "left")

In [15]:
values = ch_store_info.chw_genre_name.value_counts().index

In [16]:
all_data["province"] = all_data["chw_area_name"].apply(lambda x : x.split(" ")[0])
all_data["city"]     = all_data["chw_area_name"].apply(lambda x : x.split(" ")[1])
all_data["locality"] = all_data["chw_area_name"].apply(lambda x : x.split(" ")[2])

In [17]:
#ONE HOT Encoding!
local_series = pd.get_dummies(all_data["locality"])
city_series = pd.get_dummies(all_data["city"])
province_series = pd.get_dummies(all_data["province"])

In [18]:
genre_series = pd.get_dummies(all_data["chw_genre_name"])

In [19]:
#label_encoding_features
encode(all_data, "province")
encode(all_data, "city")
encode(all_data, "locality")
encode(all_data, "chw_genre_name")

In [20]:
del_cols = ["chw_area_name"]
all_data.drop(del_cols, axis = "columns", inplace = True)

In [21]:
#########chwiggy reserve info#######
reserve_data = pd.read_csv("chw_reserve.csv")
reserve_data.drop_duplicates()

reserve_data["visit_time"] = reserve_data["visit_datetime"].apply(lambda x : x.split()[1])
reserve_data["visit_date"] = reserve_data["visit_datetime"].apply(lambda x : x.split()[0])

reserve_data["reserve_time"] = reserve_data["reserve_datetime"].apply(lambda x : x.split()[1])
reserve_data["reserve_date"] = reserve_data["reserve_datetime"].apply(lambda x : x.split()[0])

In [22]:
reserve_data["reserve_time"] = reserve_data["reserve_time"].apply(map_time)
reserve_data["visit_time"] = reserve_data["visit_time"].apply(map_time)

In [23]:
del_cols = ["visit_datetime","reserve_datetime"]
reserve_data.drop(del_cols, axis = "columns", inplace = True)

In [24]:
reserve_visits_agg = gb_util(reserve_data,["chw_store_id", "visit_date"],"reserve_visitors","reserve_visits")
reserve_visits_agg.columns = ["chw_store_id", "date", "reserve_visitors"]
reservations_agg = gb_util(reserve_data,["chw_store_id", "reserve_date"],"reserve_visitors","reservations")
reservations_agg.columns = ["chw_store_id", "date", "reservations"]

In [25]:
all_data = all_data.merge(reservations_agg, on = ["chw_store_id", "date"], how = "left")
all_data = all_data.merge(reserve_visits_agg, on = ["chw_store_id", "date"], how = "left")

In [26]:
yom_data = pd.read_csv("yom_reserve.csv")
yom_data.drop_duplicates()

,yom_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,1/1/2016 11:00,1/1/2016 9:00,1
1,hpg_dac72789163a3f47,1/1/2016 13:00,1/1/2016 6:00,3
2,hpg_c8e24dcf51ca1eb5,1/1/2016 16:00,1/1/2016 14:00,2
3,hpg_24bb207e5fd49d4a,1/1/2016 17:00,1/1/2016 11:00,5
4,hpg_25291c542ebb3bc2,1/1/2016 17:00,1/1/2016 3:00,13
...,...,...,...,...
1048570,hpg_c20d81d9cb17891b,11/30/2016 12:00,11/27/2016 15:00,2
1048571,hpg_c89a6b67db37d5f8,11/30/2016 12:00,11/28/2016 18:00,10
1048572,hpg_ccdd56aafb8ebc9a,11/30/2016 12:00,11/26/2016 13:00,3
1048573,hpg_cd658dcb7253e28a,11/30/2016 12:00,11/29/2016 16:00,2


In [27]:
yom_data["visit_time"] = yom_data["visit_datetime"].apply(lambda x : x.split()[1])
yom_data["visit_date"] = yom_data["visit_datetime"].apply(lambda x : x.split()[0])

yom_data["reserve_time"] = yom_data["reserve_datetime"].apply(lambda x : x.split()[1])
yom_data["reserve_date"] = yom_data["reserve_datetime"].apply(lambda x : x.split()[0])

In [28]:
yom_data["reserve_time"] = yom_data["reserve_time"].apply(map_time)
yom_data["visit_time"]   = yom_data["visit_time"].apply(map_time)

In [29]:
yom_visits_agg = gb_util(yom_data,["yom_store_id", "visit_date"],"reserve_visitors","reserve_visits")
yom_visits_agg.columns = ["yom_store_id", "date", "reserve_visitors_yom"]

yom_agg = gb_util(yom_data,["yom_store_id", "reserve_date"],"reserve_visitors","reservations")
yom_agg.columns = ["yom_store_id", "date", "reservations_yom"]

In [30]:
relations = pd.read_csv("store_id_relation.csv")
all_data = all_data.merge(relations, on = ["chw_store_id"], how = "left")


In [31]:
all_data = all_data.merge(yom_agg, on = ["yom_store_id", "date"], how = "left")
all_data = all_data.merge(yom_visits_agg, on = ["yom_store_id", "date"], how = "left")

In [32]:
fill_cols = ["reservations", "reserve_visitors", "reservations_yom", "reserve_visitors_yom"]
for col in fill_cols:
    all_data[col].fillna(0, inplace = True)
all_data["reservations"] = all_data["reservations"] + all_data["reservations_yom"]
all_data["reserve_visitors"] = all_data["reserve_visitors"] + all_data["reserve_visitors_yom"]
del_cols = ["reservations_yom","reserve_visitors_yom","yom_store_id"]
all_data.drop(del_cols, inplace = True, axis = "columns")

In [33]:
def create_lags_with_gb(lag,index_cols,df, null_value = 0):
    global all_data
    df_temp = df.copy()
    df_temp["date"] = pd.to_datetime(df_temp['date']) + pd.to_timedelta(lag,unit='d')
    df_temp["date"] = pd.to_datetime(df_temp['date']).dt.strftime("%m/%d/%y")
    columns = []
    if lag != 0:
        for col in df_temp.columns.values:
            if col in index_cols:
                columns.append(col)
            else:
                columns.append(col + "_lag_" + str(lag))
        df_temp.columns = columns
    
    all_data = all_data.merge(df_temp,how="left",on=index_cols)
    del df_temp
    gc.collect()
    for col in [col for col in columns if not col in index_cols]:
        all_data[col].fillna(null_value, inplace = True)
        all_data[col] = all_data[col].astype(np.float32)
        gc.collect()

In [34]:
all_data["date"] = pd.to_datetime(all_data['date']).dt.strftime("%m/%d/%y")

In [35]:
def create_custom_lags(index_cols, col, name, op):
    global all_data
    agg = gb_util(all_data, index_cols.copy(), "visitors",name,op)
    
    agg[col] += 1
    cols = index_cols + [name + "_"+"lag"+"_"+str(1)]
    agg.columns = cols
    all_data = all_data.merge(agg,on=index_cols, how="left")
    all_data[name + "_"+"lag"+"_"+str(1)].fillna(-1, inplace = True)
    
    agg[col] -= 2
    cols = index_cols + [name + "_"+"lag"+"_"+str(-1)]
    agg.columns = cols
    all_data = all_data.merge(agg,on=index_cols, how="left")
    all_data[name + "_"+"lag"+"_"+str(-1)].fillna(-1, inplace = True)

In [36]:
lags = list(range(31, 60))
all_data["visitors_mean_past_30_2"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_past_30_2"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(15, 29))
all_data["visitors_mean_past_14_2"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_past_14_2"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(31, 45))
all_data["visitors_mean_mid_14_2"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_mid_14_2"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(-60, -30))
all_data["visitors_mean_future_30"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_future_30"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(-29, -15))
all_data["visitors_mean_fu_14"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_fu_14"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

In [37]:
lags = list(range(1, 30))
all_data["visitors_mean_past_30"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_past_30"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(1, 14))
all_data["visitors_mean_past_14"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_past_14"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(14, 30))
all_data["visitors_mean_mid_14"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_mid_14"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(-30, 0))
all_data["visitors_mean_future_30"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_future_30"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(-14, 0))
all_data["visitors_mean_fu_14"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_fu_14"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

lags = list(range(-30, -13))
all_data["visitors_mean_fu_mid_14"] = get_lag_mean(all_data,["visitors"],lags,["chw_store_id","date"])
all_data["visitors_median_fu_mid_14"] = get_lag_median(all_data,["visitors"],lags,["chw_store_id","date"])

In [38]:
for lag in range(1,30):
    all_data = create_lags(all_data,["visitors"],lag,["chw_store_id","date"])
for lag in [-1,-2,-3,-4,-5,-6-7,-14,-15,-16,-18,-20,-21,-28]:
    all_data = create_lags(all_data,["visitors"],lag,["chw_store_id","date"])
for lag in range(1,8):
    all_data = create_lags(all_data,["reservations"],lag,["chw_store_id","date"])
for lag in range(1,8):
    all_data = create_lags(all_data,["reserve_visitors"],lag,["chw_store_id","date"])    
for lag in [2,3,4,5,7,14,21,28]:
    all_data = create_lags(all_data,["median"],lag,["chw_store_id","date"])
for lag in [2,3,4,5,7,14,21,28]:
    all_data = create_lags(all_data,["mean"],lag,["chw_store_id","date"])
for lag in [-2,-5,-7,-4,-3,-21,-28]:
    all_data = create_lags(all_data,["median"],lag,["chw_store_id","date"])
for lag in [-2,-5,-7,-4,-3,-21,-28]:
    all_data = create_lags(all_data,["mean"],lag,["chw_store_id","date"])

In [39]:
agg = gb_util(all_data,["date","locality"],"visitors","local_visitors")
print("lagging local_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","locality"], agg)
    
    
agg = gb_util(all_data,["date","locality","chw_genre_name"],"visitors","local_genere_visitors")
print("lagging local_genere_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","locality","chw_genre_name"], agg)
    
    
agg = gb_util(all_data,["date","chw_genre_name"],"visitors","genere_visitors")
print("lagging genere_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","chw_genre_name"], agg)
    

agg = gb_util(all_data,["date","city"],"visitors","city_visitors")
print("lagging city_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","city"], agg)
    

agg = gb_util(all_data,["date","city","chw_genre_name"],"visitors","city_genere_visitors")
print("lagging city_genere_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","city","chw_genre_name"], agg)


agg = gb_util(all_data,["date","province","chw_genre_name"],"visitors","province_genere_visitors")
print("lagging province_genere_visitors")
for lag in range(0,8):
    create_lags_with_gb(lag,["date","province","chw_genre_name"], agg)

lagging local_visitors
lagging local_genere_visitors
lagging genere_visitors
lagging city_visitors
lagging city_genere_visitors
lagging province_genere_visitors


In [40]:
all_data = create_lags_with_name(all_data,["holiday"],1,["chw_store_id","date"],"previous_day_holiday")
all_data = create_lags_with_name(all_data,["holiday"],-1,["chw_store_id","date"],"next_day_holiday")
all_data = create_lags_with_name(all_data,["holiday"],2,["chw_store_id","date"],"previous_day_holiday_2")
all_data = create_lags_with_name(all_data,["holiday"],-2,["chw_store_id","date"],"next_day_holiday_2")

In [41]:
df_temp = all_data.loc[all_data.groupby(["chw_store_id"])["visitors"].idxmax()].copy()
df_temp = df_temp[["chw_store_id","month","week"]]
df_temp.columns = ["chw_store_id", "max_shop_visit_month", "max_shop_visit_week"]
all_data = all_data.merge(df_temp, on = "chw_store_id", how = "left")

In [42]:
df_temp = all_data.loc[all_data.groupby(["locality"])["visitors"].idxmax()].copy()
df_temp = df_temp[["locality","month","week"]]
df_temp.columns = ["locality", "max_local_visit_month", "max_local_visit_week"]
all_data = all_data.merge(df_temp, on = "locality", how = "left")

In [43]:
df_temp = all_data.loc[all_data.groupby(["chw_genre_name"])["visitors"].idxmax()].copy()
df_temp = df_temp[["chw_genre_name","month","week"]]
df_temp.columns = ["chw_genre_name", "max_genere_visit_month", "max_genere_visit_week"]
all_data = all_data.merge(df_temp, on = "chw_genre_name", how = "left")

In [44]:
df_temp = all_data.loc[all_data.groupby(["city"])["visitors"].idxmax()].copy()
df_temp = df_temp[["city","month","week"]]
df_temp.columns = ["city", "max_city_visit_month", "max_city_visit_week"]
all_data = all_data.merge(df_temp, on = "city", how = "left")

In [45]:
df_temp = all_data.loc[all_data.groupby(["city","chw_genre_name"])["visitors"].idxmax()].copy()
df_temp = df_temp[["city","chw_genre_name","month","week"]]
df_temp.columns = ["city","chw_genre_name", "max_city_genre_visit_month", "max_city_genre_visit_week"]
all_data = all_data.merge(df_temp, on = ["city", "chw_genre_name"], how = "left")

In [46]:
df_temp = all_data.loc[all_data.groupby(["locality","chw_genre_name"])["visitors"].idxmax()].copy()
df_temp = df_temp[["locality","chw_genre_name","month","week"]]
df_temp.columns = ["locality","chw_genre_name", "max_local_genre_visit_month", "max_local_genre_visit_week"]
all_data = all_data.merge(df_temp, on = ["locality", "chw_genre_name"], how = "left")

In [47]:
label_encoder = LabelEncoder()
label_encoder.fit(all_data["chw_store_id"].values)
all_data["le_id"] = label_encoder.transform(all_data["chw_store_id"])

In [48]:
scale_features = [col for col in all_data.columns.values if not col in ["chw_store_id","visit_date","le_id"]]

In [49]:
scale_features

['visitors',
 'date',
 'week_day',
 'holiday',
 'quarter',
 'week_end',
 'week',
 'month',
 'year',
 'day',
 'days',
 'month_holiday_agg',
 'month_week_holiday_agg',
 'month_num',
 'week_num',
 'median',
 'mean',
 'chw_genre_name',
 'latitude',
 'longitude',
 'province',
 'city',
 'locality',
 'reservations',
 'reserve_visitors',
 'visitors_mean_past_30_2',
 'visitors_median_past_30_2',
 'visitors_mean_past_14_2',
 'visitors_median_past_14_2',
 'visitors_mean_mid_14_2',
 'visitors_median_mid_14_2',
 'visitors_mean_future_30',
 'visitors_median_future_30',
 'visitors_mean_fu_14',
 'visitors_median_fu_14',
 'visitors_mean_past_30',
 'visitors_median_past_30',
 'visitors_mean_past_14',
 'visitors_median_past_14',
 'visitors_mean_mid_14',
 'visitors_median_mid_14',
 'visitors_mean_fu_mid_14',
 'visitors_median_fu_mid_14',
 'visitors_lag_1',
 'visitors_lag_2',
 'visitors_lag_3',
 'visitors_lag_4',
 'visitors_lag_5',
 'visitors_lag_6',
 'visitors_lag_7',
 'visitors_lag_8',
 'visitors_lag_9',

In [50]:
df = all_data.copy()

In [51]:
df = df.join(local_series)

In [52]:
df = df.join(city_series)
df = df.join(province_series)
df = df.join(genre_series)

In [53]:
df.to_csv("train_data.csv", index = False)